# Preprocesamiento de los datos

**Catálogo de valores:**

*Número_de_inclusiones:* 1 <15 ; 2 = 15–30 ; 3 = 31 – 45; 4 >45

*Agujeros:* 1 – absent; 2 – light; 3 – noticeable; 4 – great

*Lineas_de_corrosion:* 1 – absent; 2 – slight; 3 – noticeable

*Señal_quimica:* 1 – light; 2 – moderate; 3 – heavy

*Diametro_mm:* 1 < 5; 2 = 5–9.9 ; 3 = 10–19.9; 4 >20

*Arreglo_de_inclusiones:* 1 – no visible; 2 – fragmented; 3 – articulated; 4 – isolated

*Forma_de_fracturas_en_terminaciones:* 1 – blunt; 2 – pinched; 3 – straight; 4 – irregular; 5 – all types 

## Inspección de los datos

### Importar librerías

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os

### Cargar datos

In [2]:
# Load the data
df = pd.read_excel('../data/raw/coprolitos_raw.xlsx')

### Inspeccionar primero elementos de nuestros datos

In [3]:
df.head()

,Ejemplar,# inclusiones,agujeros,líneas de corrosión,señal química,diamétro (mm),arreglo de inclusiones,forma de fracturas en terminaciones
0,1 bump headed lace,4,2.0,1.0,1.0,1,2,5.0
1,2 bump headed lace,4,2.0,2.0,1.0,1,2,3.0
2,3 circular,1,2.0,1.0,1.0,1,4,5.0
3,4 circular,3,2.0,2.0,1.0,2,4,5.0
4,5 cilíndrico,1,NaN,NaN,NaN,3,1,NaN


Notamos que los valores de la columna Ejemplar contiene texto adicional, lo eliminamos para quedarnos solo con el nombre.

In [4]:
# Función Viowi

# Definir una función que realiza la transformación...
def limpiar_columna(texto):
    # Eliminar los números
    texto_sin_numeros = re.sub(r'\d+', '', texto)
    # Eliminar espacios a la izquierda y a la derecha
    texto_limpio = texto_sin_numeros.strip()
    return texto_limpio

# Aplicar la función a la columna del DataFrame
df['Ejemplar'] = df['Ejemplar'].apply(limpiar_columna)

# Imprimir los nuevos valores de la columna Ejemplar
df['Ejemplar'].unique()


array(['bump headed lace', 'circular', 'cilíndrico', 'ciclíndrico',
       'cono', 'elipsoidal', 'elongado', 'forma de pino', 'irregular',
       'forma de rosario', 'espiral', 'forma de encaje recto',
       'encaje delgado'], dtype=object)

Notamos que existe un typo en "ciclíndrico" 

In [5]:
df.loc[df['Ejemplar'] == 'ciclíndrico', 'Ejemplar'] = 'cilíndrico';

# Imprimos nuevamente los valores de la columna
df['Ejemplar'].unique()

array(['bump headed lace', 'circular', 'cilíndrico', 'cono', 'elipsoidal',
       'elongado', 'forma de pino', 'irregular', 'forma de rosario',
       'espiral', 'forma de encaje recto', 'encaje delgado'], dtype=object)

### Cambiar el nombre de las columnas

In [6]:
# Cambiar el nombre de las columnas
df = df.rename(columns={'# inclusiones': 'Numero_de_inclusiones', 'agujeros': 'Agujeros', 'líneas de corrosión': 'Lineas_de_corrosion', 'señal química': 'Señal_quimica', 'diamétro (mm)': 'Diametro_mm', 'arreglo de inclusiones' : 'Arreglo_de_inclusiones', 'forma de fracturas en terminaciones': 'Forma_de_fracturas_en_terminaciones'})

# Imprimir nuevas columnas
df.columns

Index(['Ejemplar', 'Numero_de_inclusiones', 'Agujeros', 'Lineas_de_corrosion',
       'Señal_quimica', 'Diametro_mm', 'Arreglo_de_inclusiones',
       'Forma_de_fracturas_en_terminaciones'],
      dtype='object')

### Valores pérdidos

In [7]:
df.isna().sum()

Ejemplar                               0
Numero_de_inclusiones                  0
Agujeros                               4
Lineas_de_corrosion                    4
Señal_quimica                          4
Diametro_mm                            0
Arreglo_de_inclusiones                 0
Forma_de_fracturas_en_terminaciones    4
dtype: int64

### Imputar valores pérdidos

Creamos un DataFrame solo con las filas que contienen valores pérdidos.

In [8]:
# Extraer las filas que contienen al menos un valor perdido
df_with_missing_values = df[df.isnull().any(axis=1)]

df_with_missing_values

,Ejemplar,Numero_de_inclusiones,Agujeros,Lineas_de_corrosion,Señal_quimica,Diametro_mm,Arreglo_de_inclusiones,Forma_de_fracturas_en_terminaciones
4,cilíndrico,1,NaN,NaN,NaN,3,1,NaN
6,cilíndrico,1,NaN,NaN,NaN,3,1,NaN
13,irregular,1,NaN,NaN,NaN,4,1,NaN
17,espiral,1,NaN,NaN,NaN,1,4,NaN


Para imputar vamos a utilizar la moda de los elementos que pertenecen al mismo Ejemplar. Dado que el Ejemplar "espiral" solo se encuentra una vez en nuestro Dataset, recurriremos al elemento más parecido. Tras consultarlo con la experta, sugiere que nos basemos en el ejemplar "cono". 

In [9]:
# Imputar valores perdidos con la moda de las filas que tienen el mismo valor en 'Ejemplar'
for row in df_with_missing_values.index:
    for col in df_with_missing_values.columns:
        if pd.isna(df_with_missing_values.loc[row, col]):
            ejemplar_value = df_with_missing_values.loc[row, 'Ejemplar']
            moda_value = df[df['Ejemplar'] == ejemplar_value][col].mode()
            if not moda_value.empty:
                df.loc[row, col] = moda_value[0]
            if df_with_missing_values.loc[row, 'Ejemplar'] == 'espiral':
                moda_value = df[df['Ejemplar'] == 'cono'][col].mode()
                df.loc[row, col] = moda_value[0]
            print(f'Valor nulo imputado en la fila {row} ({df.loc[row,'Ejemplar']}), columna {col} con la moda {moda_value[0]}')

Valor nulo imputado en la fila 4 (cilíndrico), columna Agujeros con la moda 2.0
Valor nulo imputado en la fila 4 (cilíndrico), columna Lineas_de_corrosion con la moda 1.0
Valor nulo imputado en la fila 4 (cilíndrico), columna Señal_quimica con la moda 1.0
Valor nulo imputado en la fila 4 (cilíndrico), columna Forma_de_fracturas_en_terminaciones con la moda 5.0
Valor nulo imputado en la fila 6 (cilíndrico), columna Agujeros con la moda 2.0
Valor nulo imputado en la fila 6 (cilíndrico), columna Lineas_de_corrosion con la moda 1.0
Valor nulo imputado en la fila 6 (cilíndrico), columna Señal_quimica con la moda 1.0
Valor nulo imputado en la fila 6 (cilíndrico), columna Forma_de_fracturas_en_terminaciones con la moda 5.0
Valor nulo imputado en la fila 13 (irregular), columna Agujeros con la moda 2.0
Valor nulo imputado en la fila 13 (irregular), columna Lineas_de_corrosion con la moda 1.0
Valor nulo imputado en la fila 13 (irregular), columna Señal_quimica con la moda 1.0
Valor nulo imputad

### Tipo de datos

In [24]:
df.dtypes

Ejemplar                                object
Numero_de_inclusiones                    int64
Agujeros                               float64
Lineas_de_corrosion                    float64
Señal_quimica                          float64
Diametro_mm                              int64
Arreglo_de_inclusiones                   int64
Forma_de_fracturas_en_terminaciones    float64
dtype: object

Dado que todos los valores son enteros representando categorías, convertimos los valores flotantes a enteros.

In [25]:
# Convertir las columnas especificadas a valores enteros, ignorando los valores perdidos
columns_to_convert = df.drop("Ejemplar", axis=1).columns
df[columns_to_convert] = df[columns_to_convert].apply(lambda x: pd.to_numeric(x, downcast='integer', errors='coerce'))

df.dtypes

Ejemplar                               object
Numero_de_inclusiones                    int8
Agujeros                                 int8
Lineas_de_corrosion                      int8
Señal_quimica                            int8
Diametro_mm                              int8
Arreglo_de_inclusiones                   int8
Forma_de_fracturas_en_terminaciones      int8
dtype: object

### Almacenar datos procesados

In [27]:
# Ruta de la carpeta donde se guardaran los datos procesadoos
folder_path = '../data/processed/'

# Verificar si la carpeta no existe y crearla si es necesario
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Ahora puedes guardar el DataFrame en formato Parquet
df.to_excel(folder_path + 'coprolitos_processed.xlsx', index=False)